In [22]:
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver import ActionChains

from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

import time
import csv

from urllib3.util import Retry
from requests import Session
from requests.adapters import HTTPAdapter
from requests.exceptions import RequestException
from random import randint

from django.contrib.gis.geos import GEOSGeometry, MultiPolygon, LinearRing, Polygon
from pprint import pprint 

In [23]:
def get_parcel_details_full(parcel_number):
    BASE_URL = 'https://xmaps.indy.gov/arcgis/rest/services/MapIndy/MapIndyProperty/MapServer/10/query?'
    #BASE_URL = 'http://xmaps.indy.gov/arcgis/rest/services/Common/CommonlyUsedLayers/MapServer/0/query?'
    #QUERY = 'where=PARCEL_C%3D+%27{parcel_number}%27&outFields=*&returnGeometry=true&f=pjson'.format(parcel_number=parcel_number)
 #   QUERY = 'where=PARCEL_C%3D%27{parcel_number}%27&text=&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=pjson'.format(parcel_number=parcel_number)
    QUERY = 'where=PARCEL_C%3D%27{parcel_number}%27&text=&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=4326&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=pjson'.format(parcel_number=parcel_number)
    s = Session()
    retries = Retry(
        total=5,
        backoff_factor=0.1,
    )
    s.mount('http://', HTTPAdapter(max_retries=retries))
    results_dict = {}

    try:
        response = s.get(BASE_URL+QUERY, timeout=30)
    except RequestException as err:
        return results_dict

    if response.status_code == 200:
       # try:
        response_json = response.json()
        FIELDS_OF_INTEREST = [
            'STNUMBER', 'PRE_DIR', 'FULL_STNAME', 'SUF_DIR', 'CITY', 'ZIPCODE',
            'FULLOWNERNAME', 'ESTSQFT', 'PROPERTY_CLASS', 'OWNERADDRESS', 'OWNERADDRESS2',
            'OWNERCITY', 'OWNERSTATE', 'OWNERZIP', 'ASSESSORYEAR_LANDTOTAL', 'ASSESSORYEAR_IMPTOTAL',
            'ASSESSORYEAR_TOTALAV', 'LEGAL_DESCRIPTION_'
        ]
        pprint(response_json)
        for field in FIELDS_OF_INTEREST:
            results_dict[field] = response_json['features'][0]['attributes'][field]
    
        #wkid = response_json['crs']['properties']['name']
        results_dict['POLY_GEOM'] = MultiPolygon(Polygon(LinearRing(response_json['features'][0]['geometry']['rings'][0], srid=2965), srid=2965), srid=2965)
        #results_dict['PNT_GEOM'] = GEOSGeometry()
    
        #lon = response_json['candidates'][0]["location"]["x"]
        #lat = response_json['candidates'][0]["location"]["y"]
        #results_dict['PNT_WKT'] = 'SRID={wkid};POINT({lon} {lat})'.format(wkid=wkid, lon=lon, lat=lat)
        #results_dict['Geocoding Accuracy']  = response_json['candidates'][0]['score']


        #except:
         #   print('Unable to retrieve details on parcel {}'.format(parcel_number,))
          #  print(results_dict)

    else:
        results_dict
    return results_dict

In [24]:
print(get_parcel_details_full('1040764'))

{'displayFieldName': 'STREET_NAME',
 'features': [{'attributes': {'ACREAGE': '0.138',
                              'ASSESSORYEAR_IMPTOTAL': 182700,
                              'ASSESSORYEAR_LANDTOTAL': 33000,
                              'ASSESSORYEAR_TOTALAV': 215700,
                              'ASSESSOR_DISTRICT': 'EAST',
                              'BLOCK': '7003',
                              'CAMAPARCELID': 218670,
                              'CITY': 'INDIANAPOLIS',
                              'COUNTY': 'MARION',
                              'ESTSQFT': 6002,
                              'FULLOWNERNAME': 'HARTLEY, CHRIS &,ANNA '
                                               'HOSTETTER',
                              'FULL_STNAME': '12TH ST',
                              'LEGAL_DESCRIPTION_': 'OTTO C WRIGHTS SUB L3',
                              'LOTNUM': ' ',
                              'MODDATE': None,
                              'NEIGHBORHOOD': '1-430R-098'